https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_for_knowledge_retrieval.ipynb

In [9]:
import openai
from projects.chatgpt.agents.LLMFunctionsAgent import LLMFunctionsAgent

# configure the client's secret key
with open('secret-key.txt', 'r') as file:
    secret_key = file.read().strip()
openai.api_key = secret_key

max_tokens = 4092

# automatically reload changes to imports
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import time
def prompt(question):
    start_time_seconds = time.time()
    print(f"asking chatgpt: {question}")
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", #"text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions accurately, without making up facts."},
            {"role": "user", "content": question}
        ],
    )
    # print(response)
    choices = response.choices
    choice = choices[0]
    message = choice['message']
    answer = message['content']
    print(answer)
    print(f"answer received in {time.time() - start_time_seconds} seconds.")

def stream_prompt(question):
    start_time_seconds = time.time()
    print(f"asking chatgpt: {question}")
    stream = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", #"text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions accurately, without making up facts."},
            {"role": "user", "content": question}
        ],
        stream=True,
    )
    for output in stream:
        choices = output['choices']
        choice = choices[0]
        delta = choice['delta']
        if 'content' in delta:
            print(delta['content'], end='')
    print()
    print(f"answer received in {time.time() - start_time_seconds} seconds.")

In [11]:
stream_prompt("""
Who is user Jason McAffee?
""")

asking chatgpt: 
Who is user Jason McAffee
I'm sorry, but I don't have access to information about specific users unless they provide that information to me.
answer received in 4.245713949203491 seconds.


In [29]:
%autoreload

llm_functions_agent = LLMFunctionsAgent(openai=openai)

def inference(question):
    llm_functions_agent.inference(question)


init


In [30]:
%autoreload

inference("""
Who is user Jason McAffee?
""")

{
  "id": "chatcmpl-8PJPAnLO0J2ZS83lwMZA2fPuhi3sj",
  "object": "chat.completion",
  "created": 1701043484,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "tool_calls": [
          {
            "id": "call_CUPhKKRWt08gncfO99ZC56XF",
            "type": "function",
            "function": {
              "name": "get_user_details",
              "arguments": "{\n  \"first_name\": \"Jason\",\n  \"last_name\": \"McAffee\"\n}"
            }
          }
        ]
      },
      "finish_reason": "tool_calls"
    }
  ],
  "usage": {
    "prompt_tokens": 123,
    "completion_tokens": 27,
    "total_tokens": 150
  }
}
get_user_details function called with arguments first_name: Jason, last_name: McAffee
added function metadata content
answer received in 4.149604082107544 seconds.
